In [2]:
from prototypes.deeplearning.dataloader.IsicDataLoader import LoadDataVectors
from prototypes.deeplearning.trainner import train_single_task
import torch
import json
from prototypes.utility.data import ProjectConfiguration
import torchvision
import os

In [3]:
weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V2
model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V2)

In [3]:
config = ProjectConfiguration("../config.json")

In [4]:
config.get_keys()

In [5]:
config.get_value("TRAIN_METADATA")

In [6]:
dataloader = LoadDataVectors(hd5_file_path=os.path.join(config.get_value("DATASET_PATH"), "train-image.hdf5"),
                             metadata_csv_path=config.get_value("TRAIN_METADATA"),
                             target_columns=["target"],
                             transform=weights.transforms())

In [ ]:
train, val = torch.utils.data.random_split(dataloader, [0.9, 0.1])

In [ ]:
len(train), len(val)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train, batch_size=config.get_value("BATCH_SIZE"), shuffle=True, num_workers=config.get_value("NUM_WORKERS"))
val_dataloader = torch.utils.data.DataLoader(val, batch_size=config.get_value("BATCH_SIZE"), shuffle=False, num_workers=config.get_value("NUM_WORKERS"))

In [7]:
# model.fc = torch.nn.Sequential(torch.nn.Linear(2048, config.get_value("NUM_CLASSES")), torch.nn.Sigmoid())
# model = model.to(device=config.get_value("TRAIN_DEVICE"))

In [ ]:
train_single_task(model=model, train_dataloader=train_dataloader, val_dataloader=val_dataloader,
                  optimizer=torch.optim.Adam(params=model.parameters(), lr=1e-4),
                  criterion=torch.nn.BCELoss(),
                  device=config.get_value("TRAIN_DEVICE"),
                  epochs=config.get_value("NUM_EPOCHS"), alpha=config.get_value("ALPHA"))

In [16]:
import onnx

model = model.to("cpu")

# export model to ONNX
dummy_input = torch.autograd.Variable(torch.randn(1, 3, 224, 224))

input_names = ["data"]
output_names = ["output"]
torch.onnx.export(model,
                  dummy_input.to("cpu"),
                  '../checkpoint_resnet50_mix_up/resnet-erasing.onnx',
                  input_names=input_names,
                  output_names=output_names,
                  opset_version=11)

In [17]:
# check created model
onnx_model = onnx.load('../checkpoint_resnet50_mix_up/resnet-erasing.onnx')
onnx.checker.check_model(onnx_model)

In [18]:
onnx_model

In [23]:
import onnxruntime
from PIL import Image


def infer_with_onnx(img_path, onnx_model):
    ort_session = onnxruntime.InferenceSession(onnx_model)
    def to_numpy(tensor):
        return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

    img = Image.open(img_path)
    img = weights.transforms([img])

    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img)}
    ort_outs = ort_session.run(None, ort_inputs)
    out_y = ort_outs[0]

    return out_y

In [24]:
infer_with_onnx("../../datasets/isic-2024-challenge/train-image/image/1.jpg", onnx_model)